### Making lookup.csv of audio paths and transcripts

In [14]:
import os
import csv
import sys
import pandas as pd

csv.field_size_limit(sys.maxsize)

def create_tsv(input_file, output_file, output2_file):
       c=0
       with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile,open(output2_file, 'w', newline='') as transcriptWithoutAudio :
        reader = csv.reader(infile, delimiter='\t')
        writer = csv.writer(outfile, delimiter='\t')
        writer2 = csv.writer(transcriptWithoutAudio, delimiter='\t')
        writer.writerow(['audio_path', 'transcript'])
        writer2.writerow(['audio_path', 'transcript'])
        for row in reader:
            audio_file = row[0]
            transcript = row[2]
            first_digit = audio_file[0]
            first_two_digits = audio_file[:2]
            audio_path = os.path.join('/media/switchblade/Windows/Users/heman/OneDrive/Desktop/BITS/Coding/Projects/SinhaVoice/audio',f'asr_sinhala_{first_digit}/asr_sinhala/data/{first_two_digits}', f'{audio_file}.flac')
            if os.path.exists(audio_path):
                writer.writerow([audio_path, transcript])
            else:
                writer2.writerow([audio_file, transcript])     
        
# Define the paths
input_file = '/media/switchblade/Windows/Users/heman/OneDrive/Desktop/BITS/Coding/Projects/SinhaVoice/tsv_files/utt_spk_text.tsv'  # File containing audio file names and transcripts
output_file = './tsv_files/lookup.tsv'  
output2_file='./tsv_files/transcriptButNoAudio.tsv'          # Output TSV file

# Create the TSV file
create_tsv(input_file, output_file,output2_file)

#### Tokenizing transcripts

In [15]:

def tokenize(text):
    return text.split()
df=pd.read_csv('./tsv_files/lookup.tsv',delimiter='\t')
df['tokens'] = df['transcript'].apply(tokenize)



#### Vocabulary Creation

In [16]:
from collections import Counter

# Flatten the list of tokenized sentences into a list of tokens
all_tokens = [token for sublist in df['tokens'] for token in sublist]

# Create a vocabulary with the frequency of each token
vocabulary = Counter(all_tokens)

# Optional: Limit the vocabulary size
vocab_size = 5000  # You can adjust this as needed
vocabulary = dict(vocabulary.most_common(vocab_size))

# Save the vocabulary to a file if needed
with open('./tsv_files/vocabulary.txt', 'w') as f:
    for token in vocabulary:
        f.write(f"{token}\n")

print(f"Vocabulary size: {len(vocabulary)}")


Vocabulary size: 5000


Mapping tokens to indices 

In [17]:
# Create a mapping from tokens to indices
token_to_idx = {token: idx for idx, token in enumerate(vocabulary.keys())}

# Convert the tokens in the dataset to indices
df['token_indices'] = df['tokens'].apply(lambda tokens: [token_to_idx.get(token, 0) for token in tokens])

# Example output
df.to_csv('./tsv_files/processed.tsv',sep='\t',index=False)
